Due to memory issues with the GPU on my laptop, Google Colab was used to train this model. You can use Colab to use a T4 GPU free of charge if you want to run this notebook there. The exact same code was run and can be viewed here:  
https://colab.research.google.com/drive/1o0dudLqBLVJ4Eq0zEIlBCL5zWu9nP_bp#scrollTo=su3veNmRFwbn

In [ ]:
import os
import pathlib
import random

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

from sklearn.metrics import f1_score, classification_report

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

try:
    import transformers
except ModuleNotFoundError:
    !pip3 install transformers datasets
finally:
    from transformers import (BertForSequenceClassification, 
                            BertTokenizerFast,
                            PreTrainedTokenizerFast,
                            DataCollatorWithPadding,
                            AdamW,
                            get_scheduler
                          )
import datasets

In [ ]:
SEED = 3141
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
# Modeling globals
BERT = 'nlpaueb/sec-bert-base'
#BERT = 'bert-base-uncased'
BSZ = 16
LR = 0.01
MOMENTUM = 0.9
DAMPENING = 0
WD = 0
NESTEROV = True
EPOCHS = 10
WARMUP_STEPS = 250
CLIP = 2.5

# Other globals
WEIGHT_DIR = './weights/financial-sentiment/'

In [ ]:
if not os.path.isdir(WEIGHT_DIR):
    pathlib.Path(WEIGHT_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if DEVICE == 'cuda':
    print('Using GPU!')
else:
    print('Using CPU!')

In [ ]:
# Import and prep data
data = datasets.load_dataset(
    "JanosAudran/financial-reports-sec",
    "small_lite",
    split="train"
)

# Downsample to train=20k and test=2k
data = data.train_test_split(train_size=0.5, test_size=0.05, seed=SEED)

In [ ]:
train_lens = []

for row in data['train']['sentence']:
    train_lens.append(len(row.split(' ')))

MAX_LEN = np.quantile(train_lens, 0.95)
print(f"95% of sentences less than {MAX_LEN} words")
MAX_LEN = int(2 ** (np.ceil(np.log2(MAX_LEN))))
print(f"Setting to nearest power of 2: {MAX_LEN}")
sns.displot(train_lens)
plt.show()

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(BERT)

def tokenize_fx(item, tokenizer: PreTrainedTokenizerFast=tokenizer):
    return tokenizer(
        text=item['sentence'],
        truncation='longest_first',
        padding='max_length',
        max_length=MAX_LEN,
        is_split_into_words=False,
        return_tensors='pt'
    )

def get_label(item, label='1d'):
    temp = pd.DataFrame(item['labels']).astype(int)
    return temp[[label]].to_dict(orient='series')

tokenized_data = data.map(tokenize_fx, batched=True)
tokenized_data = tokenized_data.map(get_label, batched=True)

tokenized_data = tokenized_data.remove_columns([
    'cik','sentence','section','filingDate',
    'docID','sentenceID','sentenceCount','labels'
])
tokenized_data = tokenized_data.rename_column('1d', 'labels')
tokenized_data.set_format('torch', device=DEVICE)

In [ ]:
# Check imbalance
prop_pos_train = tokenized_data['train']['labels'].float().mean() 
prop_pos_valid = tokenized_data['test']['labels'].float().mean()

if abs(0.5-prop_pos_train) > 0.1:
    print(f"Too much imbalance in train set {prop_pos_train,item()}")
else:
    print(f"Prop train: {prop_pos_train.item()}")
if abs(0.5-prop_pos_valid) > 0.1:
    print(f"Too much imbalance in valid set {prop_pos_valid.item()}")
else:
    print(f"Prop valid: {prop_pos_valid.item()}")

In [ ]:
train_loader = DataLoader(
    tokenized_data['train'],
    shuffle=True,
    batch_size=BSZ
)
val_loader = DataLoader(
    tokenized_data['test'],
    batch_size=BSZ
)

In [ ]:
# Validate loaders have properly shaped batches
exp_shapes = {
    'input_ids': torch.Size([BSZ, MAX_LEN]),
    'token_type_ids': torch.Size([BSZ, MAX_LEN]),
    'attention_mask': torch.Size([BSZ, MAX_LEN]),
    'labels': torch.Size([BSZ])
}

for nm, loader in zip(['train','val'],[train_loader, val_loader]):
    print(f"Checking {nm} loader...")
    for batch in loader:
        break
    for k,v in batch.items():
        act_shape = torch.tensor(v.shape)
        exp_shape = torch.tensor(exp_shapes[k])
        assert torch.equal(
            act_shape,
            exp_shape
        ), f'\tSize mismatch for {k}! Got {act_shape}, expected {exp_shape}'
    print("\tAll shapes correct!")

In [ ]:
# Model loading and other prep
NUM_LABELS = 2

model = BertForSequenceClassification.from_pretrained(
    BERT,
    num_labels=NUM_LABELS,
    problem_type='single_label_classification'
)

t_batch = {}
for k,v in batch.items():
    t_batch.update({k: v.to('cpu')})

# Check the shape of the output
out = model(**t_batch)

out_shape = torch.tensor(out.logits.shape)
assert torch.equal(
    out_shape,
    torch.tensor([BSZ, NUM_LABELS])
), f'Output shape incorrect! Got {out_shape}, expected [{BSZ},{NUM_LABELS}]'
print("Output shape good!")

In [ ]:
num_training_steps = EPOCHS * len(train_loader)
optimizer = optim.SGD(
    model.parameters(),
    lr=LR,
    momentum=MOMENTUM,
    dampening=DAMPENING,
    weight_decay=WD,
    nesterov=NESTEROV
    )
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=num_training_steps
)
print(f"Num Training Steps: {num_training_steps}")

In [ ]:
# Clear cuda memory
torch.cuda.empty_cache()

In [ ]:
# Model training
train_losses = []
val_losses = []
val_scores = []
best_loss = np.inf

pbar = tqdm(range(num_training_steps+EPOCHS*len(val_loader)))

for epoch in range(EPOCHS):
    model.to(DEVICE)
    # Training portion
    model.train()
    pbar.set_description(f"Epoch {epoch+1}: Train")
    for batch in train_loader:
        optimizer.zero_grad()
        
        out = model(**batch)
        loss = out.loss
        loss.backward()
        train_losses.append(loss.cpu())
        
        nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        optimizer.step()
        scheduler.step()
        pbar.update(1)
        
    # Validate portion
    model.eval()
    pbar.set_description(f"Epoch {epoch+1}: Validate")
    epoch_val_preds = []
    epoch_val_true = []
    epoch_val_loss = 0
    for batch in val_loader:
        with torch.no_grad():
            out = model(**batch)
        logits = out.logits.cpu()
        preds = torch.argmax(logits, dim=-1).int().numpy().tolist()
        labels = batch['labels'].cpu().numpy().tolist()
        
        epoch_val_preds += preds
        epoch_val_true += labels
        epoch_val_loss += out.loss.cpu()
        pbar.update(1)
    
    # Checkpoint model if it improves
    if epoch_val_loss < best_loss:
        print('Model improved, saving weights!')
        model.cpu().save_pretrained(WEIGHT_DIR)
        best_loss = epoch_val_loss
    
    print("Val Loss: {:.6f}".format(epoch_val_loss/len(val_loader)))
    print(classification_report(
        epoch_val_true,
        epoch_val_preds,
        zero_division=0
    ))